In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from tqdm import tqdm
from datetime import date, timedelta

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers import LSTM
from keras import callbacks
from keras import optimizers
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
path = './input/'
df_train = pd.read_csv(path+'train.csv',
converters={'unit_sales':lambda u: np.log1p(float(u)) if float(u) > 0 else 0},parse_dates=["date"])
df_test  = pd.read_csv(path + "test.csv",parse_dates=["date"])
items = pd.read_csv(path+'items.csv')
stores = pd.read_csv(path+'stores.csv')
# 类型转换
df_train['onpromotion'] = df_train['onpromotion'].astype(bool)
df_test['onpromotion'] = df_test['onpromotion'].astype(bool)

In [3]:
df_2017 = df_train.loc[df_train.date>=pd.datetime(2015,12,1)] 
del df_train

df_2017 = df_2017.merge(items, on='item_nbr', how='left')
df_2017 = df_2017.merge(stores, on='store_nbr', how='left')

In [4]:
tmp = df_2017[df_2017['date']=='2016-12-26']
tmp['date'] = '2016-12-25'
df_2017 = pd.concat([df_2017, tmp], axis=0, ignore_index=True)

In [5]:
promo_2017_train = df_2017.set_index(["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)

promo_2017_test = df_test.set_index(["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)

promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)
promo_2017 = promo_2017.astype('int')

In [6]:
df_2017 = df_2017.set_index(["store_nbr", "item_nbr", "city", "class", "date"])[["unit_sales"]].unstack(level=-1).fillna(0)
df_2017.columns = df_2017.columns.get_level_values(1) 

In [7]:
def get_date_range(df, dt, forward_steps, periods, freq='D'):
    return df[pd.date_range(start=dt-timedelta(days=forward_steps), periods=periods, freq=freq)]

In [8]:
def prepare_dataset(t2017, is_train=True):
    X = pd.DataFrame({
        # 视点前 N日促销次数
        'promo_3_2017': get_date_range(promo_2017, t2017, 3, 3).sum(axis=1).values,
        'promo_7_2017': get_date_range(promo_2017, t2017, 7, 7).sum(axis=1).values,
        'promo_14_2017': get_date_range(promo_2017, t2017, 14, 14).sum(axis=1).values,
        # 预测集一年前的16日统计销量
        "last_year_mean": get_date_range(df_2017, t2017, 365, 16).mean(axis=1).values,
        "last_year_meidan": get_date_range(df_2017, t2017, 365, 16).median(axis=1).values,
        "last_year_max": get_date_range(df_2017, t2017, 365, 16).max(axis=1).values,
        "last_year_min": get_date_range(df_2017, t2017, 365, 16).min(axis=1).values,
        # 预测集一年前的16日0销次数
        "last_year_count0": (get_date_range(df_2017, t2017, 365, 16)==0).sum(axis=1).values,
        # 预测集一年前的16日促销次数
        "last_year_promo": get_date_range(promo_2017, t2017, 365, 16).sum(axis=1).values
    })
    
    for i in range(1,8):
        # 历史平移，前 N天的销量
        X["day_{}_hist".format(i)] = get_date_range(df_2017, t2017, i, 1).values.ravel()
        
    
    for i in [3, 4, 5, 6, 7, 10, 14, 21, 30, 60, 90, 120, 140, 356]:
        for d in [0,7,14]:
            # 窗口统计，销量 diff/mean/meidan/max/min/std
            X['before_diff_{}_day_mean'.format(i)] = get_date_range(df_2017, t2017-timedelta(days=d), i, i).diff(1,axis=1).mean(axis=1).values
            X['after_diff_{}_day_mean'.format(i)] = get_date_range(df_2017, t2017-timedelta(days=d), i, i).diff(-1,axis=1).mean(axis=1).values
            X['mean_%s_decay_1' % i] = (get_date_range(df_2017, t2017-timedelta(days=d), i, i) * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values
            X['mean_%s_decay_2' % i] = (get_date_range(df_2017, t2017-timedelta(days=d), i, i) * np.power(0.7, np.arange(i)[::-1])).sum(axis=1).values
            X['mean_%s_decay_3' % i] = (get_date_range(df_2017, t2017-timedelta(days=d), i, i) * np.power(0.5, np.arange(i)[::-1])).sum(axis=1).values
            X['mean_{}_day'.format(i)] = get_date_range(df_2017, t2017-timedelta(days=d), i, i).mean(axis=1).values
            X['median_{}_day'.format(i)] = get_date_range(df_2017, t2017-timedelta(days=d), i, i).median(axis=1).values
            X['max_{}_day'.format(i)] = get_date_range(df_2017, t2017-timedelta(days=d), i, i).max(axis=1).values
            X['min_{}_day'.format(i)] = get_date_range(df_2017, t2017-timedelta(days=d), i, i).min(axis=1).values
            X['std_{}_day'.format(i)] = get_date_range(df_2017, t2017-timedelta(days=d), i, i).std(axis=1).values

        # 有/无促销的时间，销量统计
        X['mean_{}_day_haspromo'.format(i)] = get_date_range(df_2017, t2017, i, i)[get_date_range(promo_2017, t2017, i, i)==1].mean(axis=1).values
        X['median_{}_day_haspromo'.format(i)] = get_date_range(df_2017, t2017, i, i)[get_date_range(promo_2017, t2017, i, i)==1].median(axis=1).values
        X['max_{}_day_haspromo'.format(i)] = get_date_range(df_2017, t2017, i, i)[get_date_range(promo_2017, t2017, i, i)==1].max(axis=1).values
        X['min_{}_day_haspromo'.format(i)] = get_date_range(df_2017, t2017, i, i)[get_date_range(promo_2017, t2017, i, i)==1].min(axis=1).values
        X['std_{}_day_hasnopromo'.format(i)] = get_date_range(df_2017, t2017, i, i)[get_date_range(promo_2017, t2017, i, i)==1].std(axis=1).values

        X['mean_{}_day_nopromo'.format(i)] = get_date_range(df_2017, t2017, i, i)[get_date_range(promo_2017, t2017, i, i)==0].mean(axis=1).values
        X['median_{}_day_nopromo'.format(i)] = get_date_range(df_2017, t2017, i, i)[get_date_range(promo_2017, t2017, i, i)==0].median(axis=1).values
        X['max_{}_day_nopromo'.format(i)] = get_date_range(df_2017, t2017, i, i)[get_date_range(promo_2017, t2017, i, i)==0].max(axis=1).values
        X['min_{}_day_nopromo'.format(i)] = get_date_range(df_2017, t2017, i, i)[get_date_range(promo_2017, t2017, i, i)==0].min(axis=1).values
        X['std_{}_day_nopromo'.format(i)] = get_date_range(df_2017, t2017, i, i)[get_date_range(promo_2017, t2017, i, i)==0].std(axis=1).values

        # 无销量次数与促销次数
        X['count0_{}_2017'.format(i)] = (get_date_range(df_2017, t2017, i, i)==0).sum(axis=1).values
        X['promo_{}_2017'.format(i)] = get_date_range(promo_2017, t2017, i, i).sum(axis=1).values
                               
        
    for i in range(7):
        # 前 N 周平均每周 i 的销量
        for periods in [5,10,15,20]:
            steps = periods * 7
            X['before_diff_{}_dow{}_2017'.format(periods,i)] = get_date_range(df_2017, t2017, steps-i, periods, freq='7D').diff(1,axis=1).mean(axis=1).values
            X['after_diff_{}_dow{}_2017'.format(periods,i)] = get_date_range(df_2017, t2017, steps-i, periods, freq='7D').diff(-1,axis=1).mean(axis=1).values
            X['mean_{}_dow{}_2017'.format(periods,i)] = get_date_range(df_2017, t2017, steps-i, periods, freq='7D').mean(axis=1).values
            X['median_{}_dow{}_2017'.format(periods,i)] = get_date_range(df_2017, t2017, steps-i, periods, freq='7D').median(axis=1).values
            X['max_{}_dow{}_2017'.format(periods,i)] = get_date_range(df_2017, t2017, steps-i, periods, freq='7D').max(axis=1).values
            X['min_{}_dow{}_2017'.format(periods,i)] = get_date_range(df_2017, t2017, steps-i, periods, freq='7D').min(axis=1).values
            X['std_{}_dow{}_2017'.format(periods,i)] = get_date_range(df_2017, t2017, steps-i, periods, freq='7D').std(axis=1).values
        
        
    for i in range(16):
        # 未来16天是否促销日
        X["promo_{}".format(i)] = promo_2017[str(t2017 + timedelta(days=i))].values.astype(np.uint8)
        X["promo_bef_{}".format(i)] = promo_2017[str(t2017 + timedelta(days=-i))].values.astype(np.uint8)
    
    if is_train:
        y = df_2017[
            pd.date_range(t2017, periods=16)
        ].values
        return X, y
    return X

In [9]:
#【训练集】以7月5日后的16天为最后一个训练集窗口，依次向前递推14周得到14个训练窗口的训练数据
X_l, y_l = [], []
t2017 = date(2017, 7, 5)
n_range = 25
for i in tqdm(range(n_range)):
    
    X_tmp, y_tmp = prepare_dataset(t2017 - timedelta(days=7 * i))
    X_l.append(X_tmp)
    y_l.append(y_tmp)
    
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l

#【验证集】7.26
X_val, y_val = prepare_dataset(date(2017, 7, 26))
#【测试集】8.16
X_test = prepare_dataset(date(2017, 8, 16), is_train=False)

100%|██████████| 25/25 [46:12<00:00, 114.41s/it]


In [ ]:
def build_model():
    model = Sequential()
    model.add(LSTM(512, input_shape=(X_train.shape[1],X_train.shape[2])))
    model.add(BatchNormalization())
    model.add(Dropout(.2))

    model.add(Dense(256))
    model.add(Activation(activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(.1))

    model.add(Dense(256))
    model.add(Activation(activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(.1))

    model.add(Dense(128))
    model.add(Activation(activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(.05))

    model.add(Dense(64))
    model.add(Activation(activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(.05))

    model.add(Dense(32))
    model.add(Activation(activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(.05))

    model.add(Dense(16))
    model.add(Activation(activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(.05))
    model.add(Dense(1))
    
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    
    return model 

In [14]:
def build_model():
    model = Sequential()
    model.add(LSTM(128, input_shape=(X_train.shape[1],X_train.shape[2])))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(64))
    model.add(Activation(activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(16))
    model.add(Activation(activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(1))
    
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    
    return model 

In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(pd.concat([X_train, X_val, X_test]))
X_train[:] = scaler.transform(X_train)
X_val[:] = scaler.transform(X_val)
X_test[:] = scaler.transform(X_test)

X_train = X_train.fillna(0)
X_val = X_val.fillna(0)
X_test = X_test.fillna(0)

X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))
X_val = X_val.values.reshape((X_val.shape[0], 1, X_val.shape[1]))

In [ ]:
val_pred = []
test_pred = []

item_perishable_dict = dict(zip(items['item_nbr'],items['perishable'].values))
train_weight = []
val_weight = []

items_ = df_2017.reset_index()['item_nbr'].tolist() * n_range
for item in items_:
    train_weight.append(item_perishable_dict[item] * 0.25 + 1)

for i in range(16):

    print("====== Step %d ======" % (i+1))
    y_mean = y_train[:, i].mean()
    # 编译部分
    model = build_model()
    # 回调函数
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss', factor=0.5, patience=3, min_lr=0.0001, verbose=1)
    earlystopping = EarlyStopping(
        monitor='val_loss', min_delta=0.0001, patience=3, verbose=1, mode='min')
    callbacks = [reduce_lr, earlystopping]
    # 训练部分
    model.fit(X_train, y_train[:, i]-y_mean, batch_size = 4096, epochs = 50, verbose=1,
              sample_weight=np.array(train_weight), validation_data=(X_val, y_val[:, i]-y_mean), 
              callbacks=callbacks, shuffle=True)

    val_pred.append(model.predict(X_val)+y_mean)
    test_pred.append(model.predict(X_test)+y_mean)
        
# 0.279288 0.310069
# 0.278647 0.309754
# 0.27667  0.310175
# 0.276117 0.309686
# 0.274491 0.307200 sale_lgb_9

====== Step 1 ======
Train on 4308475 samples, validate on 172339 samples
Epoch 1/50
4308475/4308475 [==============================] - 169s 39us/step - loss: 0.5030 - mean_squared_error: 0.4752 - val_loss: 0.2849 - val_mean_squared_error: 0.2849
Epoch 2/50
4308475/4308475 [==============================] - 132s 31us/step - loss: 0.3411 - mean_squared_error: 0.3231 - val_loss: 0.2799 - val_mean_squared_error: 0.2799
Epoch 3/50
4308475/4308475 [==============================] - 123s 29us/step - loss: 0.3349 - mean_squared_error: 0.3174 - val_loss: 0.2809 - val_mean_squared_error: 0.2809
Epoch 4/50
4308475/4308475 [==============================] - 144s 33us/step - loss: 0.3310 - mean_squared_error: 0.3138 - val_loss: 0.2795 - val_mean_squared_error: 0.2795
Epoch 5/50
4308475/4308475 [==============================] - 139s 32us/step - loss: 0.3282 - mean_squared_error: 0.3112 - val_loss: 0.2822 - val_mean_squared_error: 0.2822
Epoch 6/50
4308475/4308475 [==============================] -

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



4308475/4308475 [==============================] - 120s 28us/step - loss: 0.3931 - mean_squared_error: 0.3717 - val_loss: 0.3463 - val_mean_squared_error: 0.3463
Epoch 8/50
4308475/4308475 [==============================] - 132s 31us/step - loss: 0.3922 - mean_squared_error: 0.3708 - val_loss: 0.3497 - val_mean_squared_error: 0.3497
Epoch 9/50
2732032/4308475 [==================>...........] - ETA: 51s - loss: 0.3904 - mean_squared_error: 0.3691

In [16]:
print("验证集 mse:", mean_squared_error(
    y_val, np.array(val_pred).transpose()[0]))

y_test = np.array(test_pred).transpose()[0]
df_preds = pd.DataFrame(
    y_test, index=df_2017.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame('unit_sales')
df_preds = df_preds.reset_index()
df_preds.columns = ['store_nbr', 'item_nbr', 'city', 'class', 'date', 'unit_sales']

submission = df_test[['id','date','store_nbr','item_nbr']].merge(df_preds, on=['date','store_nbr','item_nbr'], how='left').fillna(0)
submission['unit_sales'] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission[['id','unit_sales']].to_csv('sale_lstm_2.csv', float_format='%.4f', index=None)

# 0.3479347767794617
# 0.3436085951363862
# 0.3420840578048175
# 0.3417190536788496
# 0.3388075841917157 sale_lgb_8.csv
# 0.45825374853685075 sale_lstm_1.csv
# 0.34803971934874334 sale_lstm_2.csv

验证集 mse: 0.34803971934874334


In [2]:
sub1 = pd.read_csv('sale_lstm_2.csv')
sub2 = pd.read_csv('sale_lgb_10.csv')
sub3 = pd.read_csv('sale_wavenet_1.csv')

In [3]:
sub1['unit_sales'] = sub1['unit_sales'] * 0.2 + sub2['unit_sales'] * 0.7 + sub3['unit_sales'] * 0.1

In [4]:
sub1.to_csv('ronghe2.csv', float_format='%.4f', index=None)